## 가장 기본적인 방법


### 로컬에서 데이터 로드하기

**클라우드 셸에서 zless 명령을 이용하면 압축된 파일의 내용을 페이지 단위로 확인**

```
zless college_scorecard.csv.gz
```

- 파일은 각 컬럼의 이름을 가진 헤더행을 가지고 있음
 - 헤더 행 이후의 각 줄은 한줄의 데이터를 표현하는 값

**데이터를 빅쿼리로 로드를 위한 데이터셋 생성**

```
bq --location=US mk ch04
```

- bq명령줄은 구글 클라우드 플랫폼상의 빅쿼리 서비스를 사용하기 위한 편리한 명령어를 제공
 - 좀 더 자세히 알고 싶으면 [여기](https://cloud.google.com/bigquery/docs/quickstarts/quickstart-command-line?hl=ko)
- mk를 이용하여 지정한 프로젝트 내에 ch04 데이터셋 생성을 명령
- 데이터에 따라 리전을 다르게 지정할 수 있음
 - 리전은 저장하는 저장소의 국가 또는 위치

**파일 안의 데이터를 빅쿼리 테이블로 로드하기**

```
bq --location=US \
  load \
  --source_format=CSV --autodetect \
  ch04.college_scorecard \
  ./college_scorecard.csv.gz
```

- load는 데이터셋을 로드하는 명령
 - 원본 형식이 CSV이며 도구가 직접 스키마를 자동으로 탐지하도록 설정
 - college_scorecard 테이블을 데이터셋 ch04 안에 생성하고 로드할 데이터는 현재 디렉토리의 college_scorecard.csv.gz 파일에서 찾도록 지시


**NULL 값을 지정해 데이터를 로드**

```
bq --location=US \
  load --null_marker=NULL \
  --source_format=CSV --autodetect \
  ch04.college_scorecard \
  ./college_scorecard.csv.gz
```

- null 값을 문자열 NULL로 표현하는 방식
- bq load 명령과 함께 사용할 수 있는 모든 옵션의 목록을 보려면 bq load --help 명령을 실행

```
bq --location=US \
  load --null_marker=NULL --replace \
  --source_format=CSV --autodetect \
  ch04.college_scorecard \
  ./college_scorecard.csv.gz
```

- 기존의 테이블을 대체 한다는 옵션 --replace 설정
- 기존의 테이블에 행을 추가하려면 --replace=false 설정

**테이블 만료일 지정**

```
# 쿼리를 이용
ALTER TABLE ch04.college_scorecard
SET OPTIONS (
  expiration_timestamp =
    TIMESTAMP_ADD(CURRENT_TIMESTAMP(), INTERVAL 7 DAY),
  description="지금으로부터 7일 뒤에 만료되는 college_scorecard 테이블"
)
```

- 테이블의 만료일을 지정하여 불필요한 기간에 비용발생을 줄여줌

**빅쿼리로 로드한 데이터에서 원하는 조건의 데이터를 검색하는 쿼리**

```
# 쿼리를 이용
SELECT
  INSTNM
  , ADM_RATE_ALL
  , FIRST_GEN
  , MD_FAMINC
  , MD_EARN_WNE_P10 , SAT_AVG
FROM
  ch04.college_scorecard
WHERE
  SAFE_CAST(SAT_AVG AS FLOAT64) > 1300
  AND SAFE_CAST(ADM_RATE_ALL AS FLOAT64) < 0.2
  AND SAFE_CAST(FIRST_GEN AS FLOAT64) > 0.1
ORDER BY
  CAST(MD_FAMINC AS FLOAT64) ASC
```

- 평균 SAT 점수 1300점 이상, 입학 경쟁률 20% 미만인 대학의 이름(INSTNM)과 입학 경쟁률 및 기타 정보를 쿼리, 1차 지망으로 진학하는 학생의 비율이 10% 초과인 대학만 골라낸 후 가계 소득이 적은순으로 정렬해 문화적, 경제적 혜택을 받지 못한 학생들에게 입학 기회를 주는 대학을 검색
- where 절에 여러가지 타입 캐스팅을 사용
 - 스키마 탐지 기능이 입학 경쟁률 컬럼을 숫자로 인식하지 않음
   - 문자열로 인식된 이유는 사생활 보호를 위해 몇몇 실제 값이 아닌 privacy 값으로 대체
  